In [1]:
from stock_selector import DataFetcher, DataCleaner, DataHandler, StockSelector
import numpy as np
import pandas as pd
import sys
import os

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


In [2]:
dataFetcher = DataFetcher()
dataHandler = DataHandler()
data_file_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
base_pool = pd.read_csv(data_file_path + '//codes_map.csv')
calender = dataFetcher.get_calender_data('2014-01-01', '2023-08-01', endlevel=[(1,2), (1), (1,2), (1,2)])

In [3]:
mv_data = pd.read_csv(data_file_path + '//daily_avg_mv_data.csv')
mv_data = mv_data.set_index('wind_code')
turnoverV_data = pd.read_csv(data_file_path + '//daily_avg_turnoverV_data.csv')
turnoverV_data = turnoverV_data.set_index('wind_code')
mv_rank = DataHandler().top_rank(mv_data, 0.8)
turnoverV_rank = DataHandler().top_rank(turnoverV_data, 0.8)
dividend_rank = pd.read_csv(data_file_path + '//dividend_index_data.csv')
dividend_rank = dividend_rank.set_index('wind_code')
dividend_rank = dividend_rank.apply(lambda col: col[col==1].index).apply(pd.Series).T
stock_pool = []
for i in range(115):
    common_stocks = np.intersect1d(mv_rank.iloc[:,i].dropna(), np.intersect1d(turnoverV_rank.iloc[:,i].dropna(), dividend_rank.iloc[:,i].dropna()))
    stock_pool.append(common_stocks)
final_result = pd.DataFrame(stock_pool).T

In [5]:
payratio_data = DataFetcher().get_payratio_data(calender)
payratio_data = DataHandler().payratio_handler(payratio_data, base_pool)

In [9]:
dividend_data = DataHandler().dividend_delta_handler(DataFetcher().get_dividend_data(start_year=2011, end_year=2024),base_pool)

In [12]:
dividend_data = dividend_data.reset_index()

In [13]:
dividend_data

,wind_code,year,dividendps
0,000001.SZ,2011,NaN
1,000001.SZ,2012,NaN
2,000001.SZ,2013,0.0800
3,000001.SZ,2014,-0.1480
4,000001.SZ,2015,-0.0035
...,...,...,...
58854,873576.BJ,2019,NaN
58855,873576.BJ,2020,0.1550
58856,873576.BJ,2021,0.1900
58857,873576.BJ,2022,-0.1550
